In [1]:
# Import libraries and models
import numpy as np
import pandas as pd
from pathlib import Path
import pickle

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
)
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [2]:
# Load parsed training data
with open("outputs/parsed_training.pkl", "rb") as f:
    training = pickle.load(f)

print("Loaded", len(training), "training tasks.")

Loaded 1000 training tasks.


In [3]:
# Helper functions for grid processing
def pad_grid(grid, target_size=30):
    H, W = grid.shape
    out = -1 * np.ones((target_size, target_size), dtype=int)
    out[:H, :W] = grid
    return out

def grid_to_vector(grid):
    return pad_grid(grid, target_size=30).flatten()

In [4]:
# Prepare raw grid dataset
X_raw = []
y_raw = []

for task_id, task in training.items():
    for inp, out in zip(task["train_inputs"], task["train_outputs"]):
        X_raw.append(grid_to_vector(inp))
        y_raw.append(int(out.sum()) % 10)

X_raw = np.vstack(X_raw)
y_raw = np.array(y_raw)

print("Raw-grid dataset:", X_raw.shape, y_raw.shape)

Raw-grid dataset: (3232, 900) (3232,)


In [5]:
# Load feature data
df_features = pd.read_pickle("outputs/features_training.pkl")

In [6]:
# Prepare feature-based dataset
df_features = pd.read_pickle("outputs/features_training.pkl")

group_cols = ["task_id", "grid_role", "grid_index"]

numeric_cols = df_features.select_dtypes(include=[int, float]).columns
numeric_cols = [c for c in numeric_cols if c not in group_cols]

df_grouped = (
    df_features
    .groupby(group_cols)[numeric_cols]
    .mean()
    .reset_index()
)

mask = df_grouped["grid_role"].isin(["train_input", "train_output"])
df_train_grids = df_grouped[mask].copy()

X_feat = df_train_grids.drop(columns=group_cols).to_numpy()

y_feat = []

for _, row in df_train_grids.iterrows():

    tid  = row["task_id"]
    idx  = int(row["grid_index"])
    out_grid = training[tid]["train_outputs"][idx]
    y_feat.append(int(out_grid.sum()) % 10)

y_feat = np.array(y_feat)

print("Feature-based dataset:", X_feat.shape, y_feat.shape)

Feature-based dataset: (6464, 18) (6464,)


In [7]:
# Define machine learning models
max_iter = 5000
n_estimators = 500

models = {
    # Baselines
    "LogReg": LogisticRegression(max_iter=max_iter),
    "LinearSVM": LinearSVC(max_iter=max_iter),
    "SGD-Hinge": SGDClassifier(loss="hinge", max_iter=max_iter, tol=1e-3),

    # Kernel methods
    "RBF-SVM": SVC(kernel="rbf", C=1.0, gamma="scale", probability=False, max_iter=max_iter),

    # Bagged / Randomized trees
    "RandomForest": RandomForestClassifier(n_estimators=n_estimators),
    "ExtraTrees": ExtraTreesClassifier(n_estimators=n_estimators),

    # Boosted decision trees
    "GradientBoosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(n_estimators=n_estimators),

    # Gradient boosting (modern variant)
    "XGBoost": XGBClassifier(
        n_estimators=n_estimators,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method="hist",
        eval_metric="logloss"
    ),

    # Neural networks
    "NeuralNet": MLPClassifier(
        hidden_layer_sizes=(128,),
        activation="relu",
        solver="adam",
        max_iter=max_iter,
        random_state=0
    ),
    "NeuralNet-Deep": MLPClassifier(
        hidden_layer_sizes=(256, 128),
        activation="relu",
        solver="adam",
        max_iter=max_iter,
        random_state=0
    ),
}

In [8]:
# Train models on raw grids
Path("outputs/models").mkdir(exist_ok=True)

for name, model in models.items():

    print(f"Training RAW model: {name}")
    model.fit(X_raw, y_raw)

    with open(f"outputs/models/raw_{name}.pkl", "wb") as f:
        pickle.dump(model, f)

Training RAW model: LogReg
Training RAW model: LinearSVM


C:\Users\kvand\PycharmProjects\ARC\.venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training RAW model: SGD-Hinge
Training RAW model: RBF-SVM
Training RAW model: RandomForest
Training RAW model: ExtraTrees
Training RAW model: GradientBoosting
Training RAW model: AdaBoost
Training RAW model: XGBoost
Training RAW model: NeuralNet
Training RAW model: NeuralNet-Deep


In [9]:
# Train models on features
for name, model in models.items():

    print(f"Training FEATURE model: {name}")
    model.fit(X_feat, y_feat)

    with open(f"outputs/models/feat_{name}.pkl", "wb") as f:
        pickle.dump(model, f)

Training FEATURE model: LogReg


C:\Users\kvand\PycharmProjects\ARC\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training FEATURE model: LinearSVM
Training FEATURE model: SGD-Hinge
Training FEATURE model: RBF-SVM
Training FEATURE model: RandomForest
Training FEATURE model: ExtraTrees
Training FEATURE model: GradientBoosting
Training FEATURE model: AdaBoost
Training FEATURE model: XGBoost
Training FEATURE model: NeuralNet
Training FEATURE model: NeuralNet-Deep
